In [1]:
import exostriker.lib.RV_mod as rv
import dill
import sys

import matplotlib.ticker as mticker

In [2]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib as mpl
from matplotlib import ticker
import numpy as np
import os.path

In [3]:
mpl.rcParams['axes.linewidth'] = 2.0 #set the value globally
mpl.rcParams['xtick.major.pad']='8'
mpl.rcParams['ytick.major.pad']='2'

 

# set tick width
mpl.rcParams['xtick.major.size'] = 8
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 2

mpl.rcParams['ytick.major.size'] = 8
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 2

mpl.rcParams["font.family"] = "sans-serif"
mpl.rc('text',usetex=True)
#font = {'family' : 'normal','weight' : 'bold','size'   : 18,'sans-serif':['Helvetica']}
#mpl.rc('font', **font)

 

In [15]:
format_im = 'pdf'
dpi = 300

subsample_factor = 2

color = ['#E53e7e', "#5d1a44", "#Ea8f80", 'g']
symbol = ['o', 'D', 'o', 'o']
markersize = [6, 5, 6, 6]
alpha = [1, 1, 1, 1]
#ax1.errorbar(Epoch2, CARMENES, fmt = "v", yerr = CARMENES_yerr, color= "#E53e7e", label="CARMENES")
#ax1.errorbar(Epoch3, HIRES, fmt = "o", yerr = HIRES_yerr, color= "#Ea8f80", label="HIRES")
#ax1.errorbar(Epoch4, HARPS, fmt = "d", yerr = HARPS_yerr, color= "#5d1a44", label="HARPS")
model_color = '0.6'
model_lw = 2.0
linestyles = [':','-.','--']
fmt = ['v','o','d','o']

skip_header = 0
skip_footer = 0

In [16]:
# -------- Planet fits ------------
file_ses = open(r"/home/trifonov/exostriker-master/exostriker/Nu_Oct_plots/Nu_Oct_best_copl_stable_with_GP_ver_087.ses", 'rb')
fit = dill.load(file_ses)
file_ses.close()

def add_jitter(fit, errors, ind):

    errors_with_jitt = np.array([np.sqrt(errors[i]**2 + fit.jitt[ii]**2)  for i,ii in enumerate(ind)])
    return errors_with_jitt




In [17]:
#### Get the time series (these below are self explanatory) ########     
jd        = fit.fit_results.rv_model.jd
rvs       = fit.fit_results.rv_model.rvs
rv_err    = fit.fit_results.rv_model.rv_err
o_c       = fit.fit_results.rv_model.o_c
idset     = fit.fit_results.idset

data_o_c = fit.fit_results.rv_model.o_c - fit.gp_model_data[0]
y_model = fit.fit_results.model + fit.gp_model_curve[0]

JD_model = fit.fit_results.model_jd


In [18]:
rv_err_with_jitt = add_jitter(fit,rv_err,idset)
#print(rv_err[idset==0][0:10], rv_err_with_jitt[idset==0][0:10],fit.jitt)

In [31]:
f, ax = plt.subplots(4,1,  figsize=(20, 14), sharex=True, facecolor='w', edgecolor='k', gridspec_kw={'height_ratios': [9, 3, 3, 3]})
f.subplots_adjust(hspace=0.01, wspace=0.01)

for i in range(4):

    if i == 0:
        ax[i].plot(JD_model, y_model, linestyle='-',color="lightgray")#, lw=2.2,) 
        ax[i].set_ylabel(r'RV [m/s]',fontsize=22, rotation = 'vertical') 

        
        for j in range(4):
            ax[i].errorbar(jd[idset==j], rvs[idset==j], fmt = fmt[j], yerr = rv_err_with_jitt[idset==j], color= color[j])

        ax[i].errorbar(jd[idset==0][0], rvs[idset==0][0], fmt = fmt[0], yerr = rv_err_with_jitt[idset==0][0], color= color[0], label="CARMENES")
        ax[i].errorbar(jd[idset==1][0], rvs[idset==1][0], fmt = fmt[1], yerr = rv_err_with_jitt[idset==1][0], color= color[1], label="HIRES")
        ax[i].errorbar(jd[idset==2][0], rvs[idset==2][0], fmt = fmt[2], yerr = rv_err_with_jitt[idset==2][0], color= color[2], label="HARPS")
        ax[i].errorbar(jd[idset==2][0], rvs[idset==2][0], fmt = fmt[2], yerr = rv_err_with_jitt[idset==2][0], color= color[2], label="HARPS")

        # ax[i].set_ylim(-25,25)
        plt.setp( ax[i].get_yticklabels(), fontsize=22,weight='bold')
        plt.setp( ax[i].get_xticklabels(), fontsize=22,weight='bold')


    if i == 1:

        
        
        ax[i].plot(JD_model, fit.fit_results.fit.T[2], linestyle='-',color="lightgray")#, lw=2.2,) 
 
                    
        #ax[i].fill_between(x=x1, y1=y1, y2=y2, facecolor="#f48c42", alpha = 0.3 )#fn.mkColor(244,140,66,128))
        
        for j in range(4):
            ax[i].errorbar(jd[idset==j], o_c[idset==j]+fit.fit_results.model_data[6][idset==j], fmt = fmt[j], yerr = rv_err_with_jitt[idset==j], color= color[j])

        ax[i].set_ylabel(r'o$-$c + GP [m/s]',fontsize=18, rotation = 'vertical') 
       # ax[i].set_ylim(-15,15)
        plt.setp( ax[i].get_yticklabels(), fontsize=22,weight='bold')
        plt.setp( ax[i].get_xticklabels(), fontsize=22,weight='bold')
    
    if i == 2:
        ax[i].plot(JD_model, fit.gp_model_curve[0], linestyle='-',color="lightgray")#, lw=2.2,) 

        x1,y1 = JD_model, fit.gp_model_curve[0]+fit.gp_model_curve[2]
        x2,y2 = JD_model, fit.gp_model_curve[0]-fit.gp_model_curve[2]
                    
        ax[i].fill_between(x=x1, y1=y1, y2=y2, facecolor="#f48c42", alpha = 0.3 )#fn.mkColor(244,140,66,128))
        
        for j in range(4):
            ax[i].errorbar(jd[idset==j], o_c[idset==j], fmt = fmt[j], yerr = rv_err_with_jitt[idset==j], color= color[j])

        ax[i].set_ylabel(r'o$-$c + GP [m/s]',fontsize=18, rotation = 'vertical') 
       # ax[i].set_ylim(-15,15)
        plt.setp( ax[i].get_yticklabels(), fontsize=22,weight='bold')
        plt.setp( ax[i].get_xticklabels(), fontsize=22,weight='bold')
    
    if i == 3:
        ax[i].plot(JD_model, [0]*len(y_model), linestyle='-',color="lightgray") 

        for j in range(4):
            ax[i].errorbar(jd[idset==j], data_o_c[idset==j], fmt = fmt[j], yerr = rv_err_with_jitt[idset==j], color= color[j])

 

        ax[i].set_xlabel(r'BJD [day]',fontsize=22)
        ax[i].set_ylabel(r'o$-$c [m/s]',fontsize=18, rotation = 'vertical') 
        #ax[i].set_xlim(min(jd),max(jd))

        ax[i].locator_params(axis="x", nbins=9)
      #  ax[i].set_ylim(-15,15)
        
        plt.setp( ax[i].get_yticklabels(), fontsize=22,weight='bold')
        plt.setp( ax[i].get_xticklabels(), fontsize=22,weight='bold')
 
# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.





    ######## Legend ##############
   # handles, labels = plt.gca().get_legend_handles_labels()
    #by_label = dict(zip(labels, handles))
   # legend = ax[0].legend(by_label.values(), by_label.keys(), frameon=True, loc='best', ncol=3, 
    #                    handlelength=0.3, numpoints=1)

    legend = ax[0].legend(loc='upper left', borderaxespad=0.2, shadow=False,handlelength=0.3, frameon=True, numpoints=1,fontsize="22")
    
    # The frame is matplotlib.patches.Rectangle instance surrounding the legend.
    frame = legend.get_frame()
    frame.set_facecolor('0.90')
    # Now add the legend with some customizations.
    #legend = ax1.legend(loc='best', borderaxespad=0., shadow=False,numpoints=1)
    
    # The frame is matplotlib.patches.Rectangle instance surrounding the legend.
    #frame = legend.get_frame()
    #frame.set_facecolor('0.90')
    
    #Set the fontsize
    for label in legend.get_texts():
        label.set_fontsize('large')
    
    #for label in legend.get_lines():
    #    label.set_linewidth(2.0)  # the legend line width
    
     
    ######## Legend ##############

    
    ax[i].set_xlim(min(jd)-100,max(jd)+100)

    
    ax[i].tick_params(direction="in", which='both', top=True, bottom=True, left=True, right=True, labeltop=False, labelbottom=True)
    ax[i].ticklabel_format(style='plain')


plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False) 

#plt.show()
plt.savefig('/home/trifonov/exostriker-master/exostriker/Nu_Oct_plots/RV_plot_final.pdf', format=format_im,dpi=dpi, bbox_inches='tight' )